# Hubspot Integration

## Package imports

In the config.py file we have hubspot api key saved as string variable

In [1]:
import urllib3, json, requests
from bs4 import BeautifulSoup
import urllib.request as urllib2
import pandas as pd
import config

In [2]:
api_key =  "?hapikey=" + config.hubspot_api_key
hubspot_api_url = "http://api.hubapi.com"

## Get number of total contacts in Hubspot

Now for the purpose of tests let's check how many contacts do we have in Hubspot

In [ ]:
def getTotalNumberOfContacts():
    # First, we build the correct url

    xulr = "/contacts/v1/contacts/statistics"

    url = hubspot_api_url + xulr + api_key
    response = urllib2.urlopen(url).read()
    str_response = response.decode('utf-8')

    statistics = json.loads(str_response)
    
    print(statistics["contacts"])
    


In [ ]:
getTotalNumberOfContacts()

## Fetching leads

Now we would like to fetch all the leads that were created in the Hubspot. First to check the parameters to call I recommend to use commented methods in for loop. Then As we will see the keys and values we can adopt the names to print function of each of rows.

In [ ]:
def get_recently_created_contacts():
    
    xulr = "/contacts/v1/lists/all/contacts/recent"

    url = hubspot_api_url + xulr + api_key
    # Now we use urllib3 to open the url and read it
    #print(url)
    response = urllib2.urlopen(url).read()
    str_response = response.decode('utf-8')
    statistics = json.loads(str_response)
    df = pd.DataFrame(statistics["contacts"])
    lista = statistics["contacts"]
    
    for l in lista:
        #print(l)
        #break
        print(l["properties"],l["form-submissions"])

In [ ]:
get_recently_created_contacts()

## Fetch MQLs

To get list of all MQLs from present month we need to create smart list with specific properties. It varies in different company but for example we need to have: 
* The contact property Lifecycle stage is equal to Marketing Qualified Lead.
* The contact property Phone number is known.
* Contact is not a member of EXCLUDE FROM ALL COMMUNICATIONS.

Then we count the number of items and print it out. Also for testing purposes we print the dataframe with the items.

In [ ]:
def get_smart_list_contacts():
    count = 0
    xulr = "/contacts/v1/lists/5459/contacts/all"

    url = hubspot_api_url + xulr + api_key
    # Now we use urllib3 to open the url and read it
    #print(url)
    response = urllib2.urlopen(url).read()
    str_response = response.decode('utf-8')
    statistics = json.loads(str_response)
    df = pd.DataFrame(statistics["contacts"])
    
    for row in df['addedAt']:
        count=count+1
    
    print(count)
    print(df)
    

In [ ]:
get_smart_list_contacts()

## Saving data to MySQL

As we have this list of MQLs we now need to save them to our MySQL database.

In [4]:
import MySQLdb
db = MySQLdb.connect(host=config.mysql_ip,port=3306, user=config.mysql_user, passwd=config.mysql_password)

In [5]:
kb = db.cursor()

In [6]:
kb.execute("SHOW DATABASES")
kb.fetchall()

(('customers',),
 ('information_schema',),
 ('monthly_deals',),
 ('monthly_leads',),
 ('monthly_mqls',),
 ('mysql',),
 ('performance_schema',),
 ('phpmyadmin',),
 ('product_launch',),
 ('product_team',))

In [7]:
kb.execute("use monthly_mqls")

0